In [ ]:
import numpy, pymc3, matplotlib.pyplot, seaborn

# generate some data
t = numpy.linspace(0,2*numpy.pi,100)
y_full = numpy.cos(5*t)
# mask the data that I want to predict (requires knowledge 
#   that one might not always have at training time).
cutoff_idx = 80
y_obs = numpy.ma.MaskedArray(y_full,numpy.arange(t.size)>cutoff_idx)

# specify and train the model, used the masked array to supply only 
#   the observed data
with pymc3.Model() as model:
  grw = pymc3.GaussianRandomWalk('grw',mu=0,sd=1,shape=y_obs.size)
  y = pymc3.Normal('y',mu=grw,sd=1,observed=y_obs)
  trace = pymc3.sample(5000, cores=1)
  y_pred = pymc3.sample_ppc(trace,samples=20000,model=model)['y']
  y_pred_mean = y_pred.mean(axis=0)

  # compute percentiles
  dfp = numpy.percentile(y_pred,[2.5,25,50,70,97.5],axis=0)

  # plot actual data and summary posterior information
  pal = seaborn.color_palette('Purples')
  fig = matplotlib.pyplot.figure(dpi=100)
  ax = fig.add_subplot(111)
  ax.plot(t,y_full,c='g',label='true value',alpha=0.5)
  ax.plot(t,y_pred_mean,c=pal[5],label='posterior mean',alpha=0.5)
  ax.plot(t,dfp[2,:],alpha=0.75,color=pal[3],label='posterior median')
  ax.fill_between(t,dfp[0,:],dfp[4,:],alpha=0.5,color=pal[1],label='CR 95%')
  ax.fill_between(t,dfp[1,:],dfp[3,:],alpha=0.4,color=pal[2],label='CR 50%')
  ax.axvline(x=t[cutoff_idx],linestyle='--',color='r',alpha=0.25)
  ax.legend()
  matplotlib.pyplot.show()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [y_missing, grw]
 47%|████████████████████████████▏                               | 2588/5500 [00:14<00:15, 192.93it/s]